In [ ]:
!pip install datasets huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.0 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `lejosdeti` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/

In [ ]:
import requests
import json
import random
import time
from datasets import load_dataset

# Parameters
num_sentences = 200
seed = 45
api_key = "" # Get your API key at https://gaia-ml.com/
model_id = "d3936949-e2a3-4acc-b323-1050aafb2738"

random.seed(seed)

dataset = load_dataset("facebook/bouquet", "agr_Latn", split="dev", trust_remote_code=True)
aguaruna_sentences = [item['src_text'] for item in dataset]

selected_sentences = random.sample(aguaruna_sentences, num_sentences)

def translate_gaia(text, src_lang, tgt_lang, timeout=180):
    url = "https://latest.gaia-ml.com/api/translate/"
    headers = {
        "Content-Type": "application/json",
        "X-API-Key": api_key
    }
    data = {
        "text": text,
        "src_lang": src_lang,
        "tgt_lang": tgt_lang,
        "model_id": model_id
    }

    try:
        response = requests.post(url, headers=headers, json=data, timeout=timeout, stream=True)
        response.raise_for_status()

        final_translation = ""
        for line in response.iter_lines(decode_unicode=True):
            if line.startswith("data: "):
                try:
                    data_str = line[6:]
                    if data_str.strip():
                        event_data = json.loads(data_str)

                        if event_data.get("type") == "translation_complete":
                            final_translation = event_data.get("final_translation", "")
                            break
                        elif "final_translation" in event_data:
                            final_translation = event_data["final_translation"]
                            break
                        elif "translation" in event_data:
                            final_translation = event_data["translation"]

                except json.JSONDecodeError:
                    continue

        return final_translation if final_translation else ""

    except Exception as e:
        print(f"Translation failed: {e}")
        return ""

results = []

for i, original_aguaruna in enumerate(selected_sentences):
    print(f"\n--- Sentence {i+1} ---")
    print(f"Original Aguaruna: {original_aguaruna}")

    spanish_translation = translate_gaia(original_aguaruna, "awajun", "español")
    print(f"Spanish Translation: {spanish_translation}")

    if spanish_translation:
        results.append({
            "sentence_id": i+1,
            "original_aguaruna": original_aguaruna,
            "spanish_translation": spanish_translation
        })
    else:
        print("Failed to get Spanish translation")

with open("bouquet_translation_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\nSaved {len(results)} translation pairs to bouquet_translation_results.json")

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'facebook/bouquet' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'facebook/bouquet' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Resolving data files:   0%|          | 0/74 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/74 [00:00<?, ?it/s]


--- Sentence 1 ---
Original Aguaruna: <A:> Atumkaitjum nuwa aidau, ¿wakegagmek ataktu juju semanatin nuwanuwak ijunjami tamash?
Spanish Translation: Señoras, ¿quieren que nos volvamos a reunir esta semana?

--- Sentence 2 ---
Original Aguaruna: Nantaki juakme waitijin betek nii kanamunum, nui wajas wajig ati tusa diyawai, makichik papi wayaun.
Spanish Translation: Al levantarse, perforó la puerta justo en su cama; parado allí, considera qué será, al ver un papel que entra.

--- Sentence 3 ---
Original Aguaruna: Waamak etsegbau aents ujaaku
Spanish Translation: La gente contó la noticia rápidamente.

--- Sentence 4 ---
Original Aguaruna: <B:> Papi augku wetanmek wechattame juu tsawan minitta juwik, senchi uwajam kanag.
Spanish Translation: Cuando te enganches con la lectura del libro, no irás a ninguna parte este día, dormirás muy profundamente.

--- Sentence 5 ---
Original Aguaruna: Dui wekaesan dekatkau jegamjai Hyde Park, nui nagkaemaki wemajai a Big Ben, atsanuk nagkaikimjai Palaci

In [ ]:
!pip install -q bert-score

In [ ]:
from bert_score import score
import json
import numpy as np
from datasets import load_dataset
import pandas as pd

with open("bouquet_translation_results.json", "r", encoding="utf-8") as f:
    gaia_results = json.load(f)

aguaruna_data = load_dataset("facebook/bouquet", "agr_Latn", split="dev")
spanish_data = load_dataset("facebook/bouquet", "spa_Latn", split="dev")

df_aguaruna = aguaruna_data.to_pandas()
df_spanish = spanish_data.to_pandas()

reference_pairs = pd.merge(
    df_aguaruna[['uniq_id', 'src_text']].rename(columns={'src_text': 'aguaruna_text'}),
    df_spanish[['uniq_id', 'src_text']].rename(columns={'src_text': 'spanish_text'}),
    on='uniq_id',
    how='inner'
)

print("BERTScore Semantic Evaluation:")
print("="*60)

P_list, R_list, F1_list = [], [], []
detailed_results = []

for item in gaia_results:
    original_aguaruna = item["original_aguaruna"]
    gaia_spanish = item["spanish_translation"]

    reference_row = reference_pairs[reference_pairs['aguaruna_text'] == original_aguaruna]

    if not reference_row.empty and gaia_spanish:
        reference_spanish = reference_row.iloc[0]['spanish_text']

        P, R, F1 = score([gaia_spanish], [reference_spanish], lang="es")

        precision = float(P[0])
        recall = float(R[0])
        f1 = float(F1[0])

        P_list.append(precision)
        R_list.append(recall)
        F1_list.append(f1)

        detailed_results.append({
            "sentence_id": item["sentence_id"],
            "aguaruna": original_aguaruna,
            "reference_spanish": reference_spanish,
            "gaia_spanish": gaia_spanish,
            "precision": precision,
            "recall": recall,
            "f1": f1
        })

        print(f"\nSentence {item['sentence_id']}:")
        print(f"Aguaruna: {original_aguaruna}")
        print(f"Reference: {reference_spanish}")
        print(f"Gaia: {gaia_spanish}")
        print(f"BERTScore -> P: {precision:.4f}, R: {recall:.4f}, F1: {f1:.4f}")

if F1_list:
    print("="*60)
    print("BERTScore Summary:")
    print(f"Average Precision: {np.mean(P_list):.4f}")
    print(f"Average Recall:    {np.mean(R_list):.4f}")
    print(f"Average F1:        {np.mean(F1_list):.4f}")
    print(f"Min F1:            {np.min(F1_list):.4f}")
    print(f"Max F1:            {np.max(F1_list):.4f}")
    print(f"Std Deviation F1:  {np.std(F1_list):.4f}")
    print(f"Total sentences:   {len(F1_list)}")

    with open("bertscore_evaluation_results.json", "w", encoding="utf-8") as f:
        json.dump(detailed_results, f, ensure_ascii=False, indent=2)

    print(f"\nDetailed results saved to bertscore_evaluation_results.json")
else:
    print("No translations to evaluate")


Resolving data files:   0%|          | 0/74 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/74 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/74 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/74 [00:00<?, ?it/s]

BERTScore Semantic Evaluation:

Sentence 1:
Aguaruna: <A:> Atumkaitjum nuwa aidau, ¿wakegagmek ataktu juju semanatin nuwanuwak ijunjami tamash?
Reference: <A:> Hola chicas, ¿quieren otra noche de chicas esta semana?
Gaia: Señoras, ¿quieren que nos volvamos a reunir esta semana?
BERTScore -> P: 0.7867, R: 0.7661, F1: 0.7763

Sentence 2:
Aguaruna: Nantaki juakme waitijin betek nii kanamunum, nui wajas wajig ati tusa diyawai, makichik papi wayaun.
Reference: Se levanta y se queda frente a la puerta de su habitación, bajo la cual ve, con la misma sorpresa y miedo, que se desliza una hoja de papel.
Gaia: Al levantarse, perforó la puerta justo en su cama; parado allí, considera qué será, al ver un papel que entra.
BERTScore -> P: 0.7487, R: 0.7211, F1: 0.7346

Sentence 3:
Aguaruna: Waamak etsegbau aents ujaaku
Reference: Anuncio urgente para la población
Gaia: La gente contó la noticia rápidamente.
BERTScore -> P: 0.7346, R: 0.6862, F1: 0.7096

Sentence 4:
Aguaruna: <B:> Papi augku wetanmek 

In [ ]:
!pip install unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.2/828.2 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 36.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Fo

In [ ]:
import json
import numpy as np
from comet import download_model, load_from_checkpoint

with open("bertscore_evaluation_results.json", "r", encoding="utf-8") as f:
    bertscore_results = json.load(f)

print("Loading COMET model...")
model_path = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(model_path)

comet_data = []
for item in bertscore_results:
    comet_data.append({
        "src": item["aguaruna"],
        "mt": item["gaia_spanish"],
        "ref": item["reference_spanish"]
    })

print(f"Evaluating {len(comet_data)} translations with COMET...")

model_output = model.predict(comet_data, batch_size=8, gpus=0)
comet_scores = model_output.scores

detailed_results = []
for i, item in enumerate(bertscore_results):
    result = item.copy()
    result["comet_score"] = float(comet_scores[i])
    detailed_results.append(result)

print("\nCOMET Score Evaluation:")
print("="*60)

for i, item in enumerate(detailed_results):
    print(f"\nSentence {item['sentence_id']}:")
    print(f"Aguaruna: {item['aguaruna']}")
    print(f"Reference: {item['reference_spanish']}")
    print(f"Gaia: {item['gaia_spanish']}")
    print(f"COMET Score: {item['comet_score']:.4f}")
    print(f"BERTScore F1: {item['f1']:.4f}")

print("="*60)
print("COMET Score Summary:")
print(f"Average COMET:     {np.mean(comet_scores):.4f}")
print(f"Min COMET:         {np.min(comet_scores):.4f}")
print(f"Max COMET:         {np.max(comet_scores):.4f}")
print(f"Std Deviation:     {np.std(comet_scores):.4f}")
print(f"Total sentences:   {len(comet_scores)}")

with open("combined_evaluation_results.json", "w", encoding="utf-8") as f:
    json.dump(detailed_results, f, ensure_ascii=False, indent=2)

print(f"\nCombined results saved to combined_evaluation_results.json")

bertscore_f1 = [item['f1'] for item in detailed_results]
correlation = np.corrcoef(comet_scores, bertscore_f1)[0, 1]
print(f"Correlation between COMET and BERTScore F1: {correlation:.4f}")

Loading COMET model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Evaluating 200 translations with COMET...


Predicting DataLoader 0: 100%|██████████| 25/25 [03:22<00:00,  8.12s/it]


COMET Score Evaluation:

Sentence 1:
Aguaruna: <A:> Atumkaitjum nuwa aidau, ¿wakegagmek ataktu juju semanatin nuwanuwak ijunjami tamash?
Reference: <A:> Hola chicas, ¿quieren otra noche de chicas esta semana?
Gaia: Señoras, ¿quieren que nos volvamos a reunir esta semana?
COMET Score: 0.6687
BERTScore F1: 0.7763

Sentence 2:
Aguaruna: Nantaki juakme waitijin betek nii kanamunum, nui wajas wajig ati tusa diyawai, makichik papi wayaun.
Reference: Se levanta y se queda frente a la puerta de su habitación, bajo la cual ve, con la misma sorpresa y miedo, que se desliza una hoja de papel.
Gaia: Al levantarse, perforó la puerta justo en su cama; parado allí, considera qué será, al ver un papel que entra.
COMET Score: 0.5732
BERTScore F1: 0.7346

Sentence 3:
Aguaruna: Waamak etsegbau aents ujaaku
Reference: Anuncio urgente para la población
Gaia: La gente contó la noticia rápidamente.
COMET Score: 0.6048
BERTScore F1: 0.7096

Sentence 4:
Aguaruna: <B:> Papi augku wetanmek wechattame juu tsawan

In [ ]:
import json
import numpy as np
from sacrebleu import CHRF

with open("combined_evaluation_results.json", "r", encoding="utf-8") as f:
    results = json.load(f)

chrf = CHRF()
chrf_scores = []

for item in results:
    score = chrf.sentence_score(item["gaia_spanish"], [item["reference_spanish"]])
    chrf_scores.append(score.score)
    item["chrf_score"] = score.score

print("chrF Score Summary:")
print(f"Average chrF:      {np.mean(chrf_scores):.4f}")
print(f"Min chrF:          {np.min(chrf_scores):.4f}")
print(f"Max chrF:          {np.max(chrf_scores):.4f}")
print(f"Std Deviation:     {np.std(chrf_scores):.4f}")

with open("final_evaluation_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Results saved to final_evaluation_results.json")

chrF Score Summary:
Average chrF:      29.4870
Min chrF:          6.3199
Max chrF:          79.1521
Std Deviation:     13.9762
Results saved to final_evaluation_results.json
